## Parse cert chain
See https://cryptography.io/en/latest/x509/reference/#cryptography.x509.load_pem_x509_certificates.

Handy.

In [1]:
from cryptography import x509

In [2]:
with open("google-com-chain.pem", "rb") as pem_file:
    certs = x509.load_pem_x509_certificates(pem_file.read())

In [3]:
assert len(certs) == 3 # Expect 3 certificates in the chain.

In [4]:
for idx, cert in enumerate(certs):
    print(f"cert {idx}, Subject: {cert.subject} Issuer: {cert.issuer}")

cert 0, Subject: <Name(CN=*.google.com)> Issuer: <Name(C=US,O=Google Trust Services,CN=WE2)>
cert 1, Subject: <Name(C=US,O=Google Trust Services,CN=WE2)> Issuer: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>
cert 2, Subject: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)> Issuer: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>


## Verify leaf cert again
This time with pyca API.

Follow the code example at https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate.tbs_certificate_bytes.

In [5]:
cert_to_check = certs[0]
print(cert_to_check.subject, cert_to_check.issuer)

<Name(CN=*.google.com)> <Name(C=US,O=Google Trust Services,CN=WE2)>


In [6]:
issuer_public_key = certs[1].public_key()
issuer_public_key

See https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ec/#elliptic-curve-signature-algorithms.

In [7]:
from cryptography.hazmat.primitives.asymmetric import ec

In [8]:
issuer_public_key.verify(
    cert_to_check.signature,
    cert_to_check.tbs_certificate_bytes,
    ec.ECDSA(cert_to_check.signature_hash_algorithm),
)